<a href="https://colab.research.google.com/github/zoebatz/CAP6640_Project/blob/main/NLP_gpt_TR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sacrebleu
!pip install evaluate
!pip install rouge_score
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 512.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requi

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import openai
import pandas as pd
import numpy as np
import os
import kagglehub
from bert_score import score


from google.colab import userdata

# print(openai.__version__)
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_NLP')

In [4]:

import sacrebleu
from nltk.translate.meteor_score import meteor_score
import nltk
import evaluate

# Download necessary resources for METEOR
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but recommended
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [35]:
# import data from google drive
file_name = 'data_eng_aln'


path = "/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/"+file_name+".csv"
data = pd.read_csv(path)
data.head()



,english,albanian
0,Help me.,Më ndihmo.
1,When did you go home?,Kur shkove në shtëpi?
2,It's not effective.,Nuk është efektive.
3,I laughed.,Qesha.
4,I was really unlucky.,Une isha vertete shume i pafat.


In [36]:
len(data)

445

In [37]:
zero_shot = f""" """

one_shot = f"""
              Input: "I have a brother."
              Output: "Kam një vëlla."
"""

three_shot = f"""
              Input: "It's a beautiful picture."
              Output: "Èshte fotografi e bukur."
              Input: "I want to say something to you."
              Output: "Dua te ju them dicka."
              Input: "You know it's not true."
              Output: "Ju e dini ge nuk eshte e vërtete."
"""


In [38]:
# text classification with GPT

def translate_text(text):
    try:
        # Call GPT-3.5 Turbo
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a translator assistant who translates from English to Albanian. Given a phrase in English, repsond with the translated phrase in Albanian."},
                {"role": "assistant", "content": shot},
                {"role": "user", "content": text}
            ]
        )
        # Extract and return the response
        translation = response.choices[0].message.content.strip()
#       print(response.choices[0].message.content)
        return translation
    except Exception as e:
        return f"Error: {e}"

In [39]:
# adjust for desired prompt / model / data size
test_size = len(data)   # can adjust for smaller testing

function = translate_text
shot_name = "three_shot"      # change for zero / one / three shot
model_name = "gpt-3.5-turbo"  # change for different models

In [40]:

prediction = []

if shot_name == "one_shot":
  shot = one_shot
elif shot_name == "three_shot":
  shot = three_shot
else:
  shot = zero_shot



def translate(data):
  count = 0
  for i, texts in enumerate(data.english[:test_size]):
    result = function(texts)
    count += 1
    if count % 100 == 0:
      print(count)

    prediction.append(result)



In [41]:
translate(data)


100
200
300
400


In [42]:
len(prediction)

445

In [43]:
# albanian to list
reference = data.albanian[:test_size].tolist()


In [44]:
file_name

'data_eng_aln'

In [45]:
shot_name


'three_shot'

In [46]:
size = len(prediction)
size

445

In [49]:
# view metrics and save
def view_and_save_metrics(prediction, reference):
  # BLEU
  bleu = sacrebleu.corpus_bleu(prediction, [reference])
#  print(f"BLEU: {bleu.score}")

  # Tokenize sentences for METEOR
  prediction_tokens = [nltk.word_tokenize(pred) for pred in prediction]
  reference_tokens = [[nltk.word_tokenize(ref)] for ref in reference]  # Note the double list for reference

  # METEOR (sentence-level average)
  meteor_scores = [
      meteor_score(ref_tokens, pred_tokens)
      for ref_tokens, pred_tokens in zip(reference_tokens, prediction_tokens)
  ]
  avg_meteor = sum(meteor_scores) / len(meteor_scores)

  # ROUGE
  rouge = evaluate.load('rouge')
  rouge_scores = rouge.compute(predictions=prediction, references=reference)
#  print(f"ROUGE: {rouge_scores}")

  # BERT Score
  P_, R_, F1 = score(prediction, reference, lang="el")
  # print(f"Avg BERTScore F1: {F1.mean().item():.4f}")
  bert_score = F1.mean().item()

  metrics = {
      "bleu": bleu.score,
      "meteor": avg_meteor,
      "bert_score": bert_score,
      "model_name": model_name,
      "dataset": file_name,
      "shot": shot_name,
      "size": size
  }

  metrics.update(rouge_scores)
  '''
  # Print all metrics
  print("\n--- Evaluation Metrics ---")
  print(f"BLEU:   {bleu.score:.4f}")
  print(f"METEOR: {avg_meteor:.4f}")
  for k, v in rouge_scores.items():
      print(f"{k.upper()}: {v:.4f}")
  '''

  # save metrics to csv file
  metrics_df = pd.DataFrame([metrics])
  metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/metrics/translation/metrics_'+model_name+'_'+file_name+'_'+shot_name+'_'+str(size)+'.csv', index=False)


In [50]:
view_and_save_metrics(prediction, reference)